In [1]:
# Start writing code here...
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import igraph

In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/lacemaker/anaconda3/envs/data_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor rele

This notebook contains template with example of data usage and loading (from prepared csv files, which were produced from original .xlsx file with the help of pandas)

In [2]:
if not ".." in sys.path:
    sys.path.insert(0, "..")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
os.listdir("../data")

['.DS_Store',
 'description.txt',
 'input_wagon.xlsx',
 'metrics.csv.gz',
 'orders7.csv.gz',
 'orders8.csv.gz',
 'output_wagon.xlsx',
 'sources.csv.gz',
 'standplaces.csv.gz',
 'wagon.zip',
 'wagon_mode_compat.csv.gz']

In [5]:
orders7 = pd.read_csv("../data/orders7.csv.gz")
orders8 = pd.read_csv("../data/orders8.csv.gz")
wagon_mode_compat = pd.read_csv("../data/wagon_mode_compat.csv.gz") # +
sources = pd.read_csv("../data/sources.csv.gz") # +
standplaces = pd.read_csv("../data/standplaces.csv.gz")
metrics = pd.read_csv("../data/metrics.csv.gz") #+

In [6]:
# load data from file (this gets checked and reloaded because of autoreload magic above)
from utils.abstractions import *

In [34]:
env = WorldModelEnv(
    orders=orders7,
    wagon_mode_compat=wagon_mode_compat,
    sources=sources, metrics=metrics)

In [35]:
env

In [36]:
def get_actions(env):
    """Custom strategy.
    
    Accepts WorldModelEnv object
    
    Returns dict: wagon_id -> (start, end, order_num, duration, tariff)
    """
    my_actions = dict()
    starts = [e for e in env.available_orders.es]
    starts = sorted(starts, key=lambda x: x['penalty'])
    starts = [(e.source, e.target, e['order_num'], e['wagon_model'], e) for e in starts]
    planned = set()
    for s, e, order, wm, ee in starts:
        sel = env.world_state.es.select(_source=s, _target=e)
        try:
            road_sel = env.roads_graph.es.select(_source=s, _target=e)
        except:
            road_sel = []
        if len(sel) == 0 or len(road_sel) == 0:
            continue
        edge = sel[0]
        road_sel = road_sel[0]
        move_cost = np.min([v for k, v in road_sel['info']])
        duration = ee['dur']
        min_unit = ee['min_unit']
        max_unit = ee['max_unit']
        tariff = ee['tariff'][0]
        wms = np.unique([e.target for e in env.wagon_graph.es.select(_source=wm)])
        start_wagons = [env.wagons[i] for i in edge.source_vertex['info'].wagons]
        start_wagons = [sx for sx in start_wagons if sx.wm_index in wms]
        start_wagons = [w for w in start_wagons if w.index not in planned]
        if len(start_wagons) == 0:
            break
        #print(start_wagons)
        
        # penalty = ee['penalty']
        #print(ee)
        if len(start_wagons) < min_unit:
            continue
        wagon_ids = [w.index for w in start_wagons][:max_unit]
        for i in wagon_ids:
            planned.add(i)
            my_actions[i] = (s, e, order, duration, tariff)
    return my_actions
    #break

In [37]:
max_day = np.unique(orders7.StartDate).max()

What step() method does:

0. validate (? ignore!)
1. add moves, move these wagons from stations to routes, compute tariff
2. inc current day
3. compute costs for the wagons which are at the stations
4. check for the newly arrived wagons: move them to destination stations, mark order as done, add profit



In [38]:
costs_n_profits = []
while env.current_step <= max_day:
    # print(env.current_step)
    my_actions = get_actions(env)
    move_cost, stay_cost, arrival_profit = env.step(my_actions)
    costs_n_profits.append((move_cost, stay_cost, arrival_profit))

In [70]:
# collect results and compute profits!
orders = set()
stay_cost = []
orders_profit = dict()
order_wagons = dict()
for move, stay, arrival in costs_n_profits:
    for k, v in move.items():
        orders.add(k)
        orders_profit[k] = np.sum(v)
        order_wagons[k] = len(v)
    stay_cost.append(np.sum(list(stay.values())))
#len(costs_n_profits)

In [71]:
profits = np.sum(list(orders_profit.values()))
profits - np.sum(stay_cost)

-2664766608

## TODO

- move idle wagons to the stations where cost is minimal
